In [1]:
from tqdm import tqdm
import os
from datasets import load_dataset, concatenate_datasets
import json
import re
from copy import deepcopy

import asyncio
import math
import numpy as np
import random
import pandas as pd
from ollama import AsyncClient

from random import sample
from collections import Counter
from openai import OpenAI

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Utils

In [24]:
def save_to_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
        
def load_from_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)

In [2]:
client = OpenAI(
    # Defaults to os.environ.get("OPENAI_API_KEY")
)

def chat_openai(content,image_path, system_prompt=None):
    messages = []
    if system_prompt:
        system_message = {'role': 'system', 'content': system_prompt}
        messages.append(system_message)
    message = {'role': 'user', 'content': content}
    messages.append(message)
    if image_path != None:
        message.update({'images': [image_path]})
    chat_completion = client.chat.completions.create(model="gpt-4o",messages=messages )
    return chat_completion.choices[0].message.content

In [46]:
client_ollama = AsyncClient(
  host='http://192.168.1.5:11434',
)

async def chat(content,image_path, system_prompt=None):
    messages = []
    if system_prompt:
        system_message = {'role': 'system', 'content': system_prompt}
        messages.append(system_message)
    message = {'role': 'user', 'content': content}
    messages.append(message)
    if image_path != None:
        message.update({'images': [image_path]})
    response = await client_ollama.chat(model='llama3.3', messages=messages,options={"temperature":0.2})
    return response['message']['content']

In [28]:
def get_prompt(doc, prompt, categories=None):
    question=doc['question']
    options=doc['options']
    if not categories:
        return '[Question]\n'+question+'\nOptions: '+options+'\n[End of Question]\n'+prompt
    return '[Question]\n'+question+'\nOptions: '+options+'\n[End of Question]\n'+'[Categories]\n'+categories+'\n[End of Categories]\n'+prompt

# Locura

In [5]:
task_name = 'AgriExam'

dataset_name = "parquet"
data_files = {
    "dev": "/workdir/important_datasets/AGRIVQA/"+task_name+"/dev-00000-of-00001.parquet",
    "test": "/workdir/important_datasets/AGRIVQA/"+task_name+"/test-00000-of-00001.parquet",
    "validation": "/workdir/important_datasets/AGRIVQA/"+task_name+"/validation-00000-of-00001.parquet"
}
split = "validation"

dataset = load_dataset(dataset_name,data_files=data_files)

Generating dev split: 16 examples [00:00, 2091.47 examples/s]
Generating test split: 624 examples [00:00, 37828.03 examples/s]
Generating validation split: 5613 examples [00:00, 56699.71 examples/s]


In [6]:
dataset

DatasetDict({
    dev: Dataset({
        features: ['id', 'question', 'options', 'explanation', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'img_type', 'answer', 'topic_difficulty', 'question_type', 'subfield', 'metadata'],
        num_rows: 16
    })
    test: Dataset({
        features: ['id', 'question', 'options', 'explanation', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'img_type', 'answer', 'topic_difficulty', 'question_type', 'subfield', 'metadata'],
        num_rows: 624
    })
    validation: Dataset({
        features: ['id', 'question', 'options', 'explanation', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'img_type', 'answer', 'topic_difficulty', 'question_type', 'subfield', 'metadata'],
        num_rows: 5613
    })
})

In [8]:
# Load dataset with splits

# Combine (concatenate) splits into a single dataset
combined_dataset = concatenate_datasets([dataset['dev'], dataset['test'], dataset['validation']])

# Now combined_dataset contains data from all splits
print(combined_dataset)


Dataset({
    features: ['id', 'question', 'options', 'explanation', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'img_type', 'answer', 'topic_difficulty', 'question_type', 'subfield', 'metadata'],
    num_rows: 6253
})


In [9]:
dataset=combined_dataset
dataset

Dataset({
    features: ['id', 'question', 'options', 'explanation', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'img_type', 'answer', 'topic_difficulty', 'question_type', 'subfield', 'metadata'],
    num_rows: 6253
})

In [11]:
prompt = '''Try to assign one category from the list of categories to the previous question, if the question is not related to any of them create a new category. 
This question is taken from an agronomy exam.
Your answer should start with one line containing only the category name.'''

In [13]:
print(get_prompt(dataset[10], prompt))

[Question]
Pink boll worm is a serious pest of?
Options: ['Mustard', 'Cotton', 'Gram', 'Soybean']
[End of Question]
Try to assign one category from the list of categories to the previous question, if the question is not related to any of them create a new category. 
This question is taken from an agronomy exam.
Your answer should start with one line containing only the category name.


In [15]:
ds = sample(list(dataset), 2000)

In [264]:
# , so try to be more specific than 'Agriculture' if it is a subfield.

In [277]:
categories = {
    'Agricultural Engineering',
    'Animal Science',
    'General Agriculture',
    'Agricultural Genetics',
    'Agricultural Health',
    'Agricultural History',
    'Plant Breeding',
    'Soil Science',
    'Agricultural Statistics',
    'Wordplay and Analogies',
    'Economics',
    'Geography and History',
    'Geology',
    'Geometry',
    'Health Science',
    'Leadership and Management',
    'Math Concepts',
    'Mathematics',
    'Sports Knowledge'
}


In [323]:
categories = {'Agricultural Economics',
 'Agricultural Education',
 'Agricultural Engineering',
 'Agricultural Genetics',
 'Agricultural Geography',
 'Agricultural Health',
 'Agricultural History',
 'Agricultural Mathematics',
 'Agricultural Science',
 'Agricultural Statistics',
 'Anatomy',
 'Animal Science',
 'Biochemistry',
 'Economics',
 'English Language',
 'Environmental Science',
 'General Agriculture',
 'General Knowledge',
 'Geography and History',
 'Geology',
 'Geometry',
 'Health Science',
 'Leadership and Management',
 'Math Concepts',
 'Mathematics',
 'Plant Biology',
 'Plant Breeding',
 'Social Stratification',
 'Soil Science',
 'Sports Knowledge',
 'Wordplay and Analogies'}

In [ ]:

#categories = set()
for doc in tqdm(dataset):
    if not categories:
        string_categories='[]'
    else:
        string_categories=str(list(categories))
    response = await chat(get_prompt(doc,prompt,string_categories),None)
    try:
        category = response.split('\n')[0]
        if category not in categories:
            print(f"Category: {category} | Question: {doc['question']} | Options: {doc['options']}")
        categories.add(category)
    except:
        print('Gnappows')

  4%|▎         | 230/6253 [00:32<14:16,  7.03it/s]

Category: Environmental Science | Question: Ozone concentration in the atmosphere is reduced by? | Options: ['CO2 and O2', 'H2S and CH4', 'CFC and Cl2', 'N2 and N2O']


100%|██████████| 6253/6253 [15:17<00:00,  6.82it/s]


In [324]:
from collections import defaultdict
categories_dict = defaultdict(list)

for doc in tqdm(dataset):
    string_categories=str(list(categories))
    response = await chat(get_prompt(doc,prompt,string_categories),None)
    try:
        category = response.split('\n')[0]
        if category not in categories:
            print(f"Category: {category} | Question: {doc['question']} | Options: {doc['options']}")
        else:
            categories_dict[category].append(doc)
    except:
        print('Gnappows')

 41%|████      | 2555/6253 [06:16<09:07,  6.76it/s]

Category: Computer Security | Question: Which of the following is a threat for electronic payment systems? | Options: ['Computer Virus', 'Computer Worms', 'Trojan Horses', 'All of the above']


100%|██████████| 6253/6253 [15:18<00:00,  6.81it/s]


In [332]:
for key in categories_dict:
    print(key,len(categories_dict[key]))
    print()
    for i in range(3):
        print(categories_dict[key][i]['question'])
    print()

General Knowledge 56

Japanese style of gardens also known as?
FOB in international trade stands for?
In Indians context meaning of the term extension’ is transfer of technology while in USA it is?

General Agriculture 1240

Identify the correct order of the following methods of irrigation as per their water-use efficiency from the maximum to minimum efficiency.?
The insect, which attacks potato both in field as well as in storage is?
Disc Harrow which type tillage instrument?

Animal Science 167

A ______ dry matter intake100 kg body weight is recommended for breeding bull?
Which Fish has the highest protein content?
What is the Average Gestation period of buffalo? / 

Soil Science 394

Which of the following is the CORRECT sequence of soil-water erosion?
What is the total number of soil orders?
The most important feature of soil taxonomy is that it is based upon?

Plant Breeding 967

Synthetic seeds are…..?
Sundaram is a very high yielding clone of?
A variety developed by mixing the 

In [373]:
prompt_agronomy = '''Evaluate the exam question provided and determine if it pertains to the field of agronomy. Please begin your response with a single word: 'Yes' if the question pertains to or is relevant in the context of agronomy, or 'No' if it is not.'''

In [ ]:
dataset_light = list(dataset)[:200]

In [374]:
agronomy_dict = {'Yes': [], 'No': []}

for doc in tqdm(dataset):
    response = await chat(get_prompt(doc, prompt_agronomy),None)
    if response.lower().startswith('yes'):
        agronomy_dict['Yes'].append(doc)
    elif response.lower().startswith('no'):
        agronomy_dict['No'].append(doc)        
    else:
        print(f"Question: {doc['question']} | Options: {doc['options']}")

100%|██████████| 6253/6253 [32:00<00:00,  3.26it/s] 


In [377]:
print(len(agronomy_dict['No']))

1670


In [384]:
no = sample(agronomy_dict['No'], 40)
for doc in no:
    print(f"Question: {doc['question']} | Options: {doc['options']}")

Question: Which one of the following compounds forms the backbone of fats and oils? | Options: ['Glycerol', 'Glucose', 'Palmitiacid', 'Amino alcohol']
Question: National Botanical Research Institute is located at? | Options: ['New Delhi', 'Lucknow', 'Vadodara', 'Chennai']
Question: Blood group system was discovered by? | Options: ['Camillo Golgi', 'Ernst Haeckel', 'Thomas Cooley', 'Karl Landsteiner']
Question: “The first task of this Assembly is to free India through a new Constitution, to feed the starving people and to cloth the naked masses and to give every Indian the fullest opportunity to develop himself according to his capacity.” This quotation was given by? | Options: ['Pandit Jawaharlal Nehru', 'Dr. B. R. Ambedkar', 'Dr. S. Radha Krishnan', 'Dr. Rajendra Prasad']
Question: Nucleic acids are? | Options: ['Micro molecular compounds', 'Macro molecular compounds', 'Micro and macro molecular compounds', 'Non-molecular compounds']
Question: Watershed is a? | Options: ['Hydrological

In [383]:
prompt_categories = '''Try to assign to the previous question one category from the list of categories, if the question is not related to any of them create a new category. 
This question is taken from an agronomy exam.
Your answer should start with one line containing only the category name.'''

In [386]:
categories = {'Agricultural Economics',
 'Agricultural Education',
 'Agricultural Engineering',
 'Agricultural Genetics',
 'Agricultural Geography',
 'Agricultural Health',
 'Agricultural History',
 'Agricultural Mathematics',
 'Agricultural Science',
 'Agricultural Statistics',
 'Anatomy',
 'Animal Science',
 'Biochemistry',
 'Economics',
 'English Language',
 'Environmental Science',
 'General Agriculture',
 'General Knowledge',
 'Geography and History',
 'Geology',
 'Geometry',
 'Health Science',
 'Leadership and Management',
 'Math Concepts',
 'Mathematics',
 'Plant Biology',
 'Plant Breeding',
 'Social Stratification',
 'Soil Science',
 'Sports Knowledge',
 'Wordplay and Analogies'}

In [398]:
categories = {
    'Agricultural Economics',
    'Agricultural Education',
    'Agricultural Engineering',
    'Genetics and Breeding',
    'Sustainable Farming',
    'Agricultural Health',
    'Agricultural History',
    'Agricultural Geography',
    'Botany',
    'Data Science in Agriculture',
    'Climate and Agriculture',
    'Animal Science',
    'Biochemistry in Agriculture',
    'Environmental Economics',
    'Farm Management',
    'Food Science',
    'Irrigation and Water Management',
    'Plant Protection',
    'Precision Agriculture',
    'Rural Sociology',
    'Soil Science',
    'Crop Production',
    'Weed and Pest Management'
}

In [396]:
from collections import defaultdict

#categories = set()
categories_dict = defaultdict(list)

for doc in tqdm(agronomy_dict['Yes']):
    if not categories:
        string_categories='[]'
    else:
        string_categories=str(list(categories))
    response = await chat(get_prompt(doc,prompt_categories,string_categories),None)
    try:
        category = response.split('\n')[0]
        if category not in categories:
            print(f"Category: {category} | Question: {doc['question']} | Options: {doc['options']}")
        #categories.add(category)
        categories_dict[category].append(doc)
    except:
        print('Gnappows')

  5%|▌         | 245/4583 [00:34<10:41,  6.76it/s]

Category: Food Science in Agriculture | Question: The Indian name for clarified butterfat is…? | Options: ['(A) cream', '(B) buttermilk', '(C) dahi', '(D) ghee']


  6%|▌         | 264/4583 [00:36<09:52,  7.29it/s]

Category: Agriculture | Question: Golden revolution is related to? | Options: ['Dairy', 'Pulses', 'Agriculture', 'Horticulture']


 18%|█▊        | 839/4583 [01:55<08:18,  7.52it/s]

...

 91%|█████████ | 4166/4583 [09:38<00:59,  7.06it/s]

Category: Plant Physiology | Question: Study of the plant processes is called? | Options: ['Biotechnology', 'Plant physiology', 'Plant molecular biology', 'Ecology']


 94%|█████████▍| 4324/4583 [10:01<00:35,  7.31it/s]

Category: Plant Anatomy | Question: The cork in dicotyledonous plants is formed by? | Options: ['Phellogen', 'Phelloderm', 'Phellum', 'Cambium']


100%|██████████| 4583/4583 [10:36<00:00,  7.20it/s]


In [407]:
for key in categories_dict:
        
        print(key, len(categories_dict[key]))

Irrigation and Water Management 152
Animal Science 167
Soil Science 633
Plant Protection 503
Genetics and Breeding 927
Agricultural Engineering 81
Geography in Agriculture 43
Climate and Agriculture 56
Crop Production 1296
Agricultural Economics 170
Agricultural Health 5
Data Science in Agriculture 19
Biochemistry in Agriculture 152
Agricultural History 81
Environmental Economics 7
Precision Agriculture 19
Weed and Pest Management 111
Food Safety 14
Agricultural Education 45
Farm Management 54
Sustainable Farming 25
Food Science in Agriculture 1
Agriculture 1
Plant Anatomy 5
Plant Physiology 7
Community Ecology 1
Agricultural Geography 1
Rural Sociology 4
Botany 1
Crop Ecology 1
Plant Biology 1


In [406]:
for c in categories_dict['Data Science in Agriculture']:
    #agronomy_dict['Yes'].remove(c)
    print(c['question'])

The value of r (correlation coefficient) ranges between?
Which test is used to test the significance of the difference between two means?
Which of the following is ideal measure of disperson?
The correlation coefficient is used to determine?
Coefficient of variation is expressed as percentage of?
Analysis of variance is a statistical method of comparing the of several populations?
What will be the error d.f. for a R.B.D. field experiment with 8 treatments and 3 replications?
Comprehension:
The table given here shows the production of five types of tractors by a company in the year 2012 to 2017. Study the table and answer the questions that follow
Table: Production of Tractors by a Company
Table:
| Company /Year | 2012 | 2013 | 2014 | 2015 | 2016 | 2017 | Total |
|---------------|------|------|------|------|------|------|-------|
| A | 8 | 20 | 16 | 17 | 21 | 6 | 88 |
| B | 16 | 10 | 14 | 12 | 12 | 14 | 78 |
| C | 21 | 17 | 16 | 15 | 13 | 8 | 90 |
| D | 4 | 6 | 10 | 16 | 20 | 31 | 87 |


# Categorize Agronomy questions

## Load questions that are about agronomy

In [29]:
dataset = load_from_json('/workdir/agriexam_category.json')
dataset

[{'id': 'dev__agriexam_2',
  'question': 'Identify the correct order of the following methods of irrigation as per their water-use efficiency from the maximum to minimum efficiency.?',
  'options': "['Continuous flowing> Flooding >Cablegation> Bubbler', 'Bubbler >Cablegation> Flooding > Continuous flowing', 'Cablegation> Flooding » Bubbler > Continuous flowing', 'Bubbler> Flooding > Continuous flowing Cablegation']",
  'explanation': None,
  'image_1': None,
  'image_2': None,
  'image_3': None,
  'image_4': None,
  'image_5': None,
  'img_type': '',
  'answer': 'B',
  'topic_difficulty': '3',
  'question_type': 'multiple-choice',
  'subfield': '{subfield}',
  'metadata': '{"source": "AgriExam", "author": "AgriExam", "license": "", "url": "https://www.agriexam.com/agronomy-jrf-2020", "language": "English", "verbose_answer": "Bubbler >Cablegation> Flooding > Continuous flowing"}'},
 {'id': 'dev__agriexam_3',
  'question': 'A ______ dry matter intake100 kg body weight is recommended for 

## Establish possible categories

In [145]:
categories = [
    # Agricultural sub-topic
    'Agricultural Economics',
    'Agricultural Education',
    'Agricultural Engineering',
    'Agricultural Health',
    'Agricultural History',
    'Agricultural Geography',
    'Agricultural Extension',
    
    # Too specific / to eliminate
    'Data Science in Agriculture',
    'Precision Agriculture',
    'Rural Sociology',
    
    # Environment
    'Sustainability and Ecology',
    'Climate and Agriculture',
    'Forestry', 
    
    # Farming
    'Animal Science',
    'Farm Management',
    
    # Agriculture
    'Fertilizers',
    'Crop Nutrient Management',
    'Irrigation and Water Management',
    'Food Science',
    'Soil Science',
    'Crop Production',
    
    #  Plant Disease
    'Pests and Weeds Management',
    'Plant Pathology',
    'Plant Protection',
    
    # Plant Science / Scienze
    'Entomology',
    'Microbiology',
    'Plant Biochemistry',
    'Genetics, Breeding and Seeds Management',
    'Horticulture and Ornamental Plants',
    'Plant Anatomy',
    'Plant Physiology',
    'Plant Taxonomy and Scientific Naming',
    'Plant Science'
]

In [146]:
def list_to_string(categories):
    categories_string=''
    for i, category in enumerate(categories):
        categories_string += f'{str(i+1)}. {category}\n'
    return categories_string

In [147]:
from random import shuffle
shuffle(categories)
system_prompt = f'''You are an expert in agronomy, skilled at categorizing questions based on their topics. I will provide you with a questions from an agronomy exam. Your task is to assign one of the following categories to the question: 

{list_to_string(categories)}

Carefully review the question's content and context to determine the most appropriate category it belongs to. Select the most specific category available from the list. Your answer should start with a single line containing only the name of the chosen category.
'''

In [148]:
print(system_prompt)

You are an expert in agronomy, skilled at categorizing questions based on their topics. I will provide you with a questions from an agronomy exam. Your task is to assign one of the following categories to the question: 

1. Microbiology
2. Plant Science
3. Plant Pathology
4. Agricultural Education
5. Plant Physiology
6. Genetics, Breeding and Seeds Management
7. Entomology
8. Irrigation and Water Management
9. Farm Management
10. Crop Nutrient Management
11. Agricultural Health
12. Data Science in Agriculture
13. Climate and Agriculture
14. Fertilizers
15. Plant Anatomy
16. Agricultural Geography
17. Precision Agriculture
18. Plant Protection
19. Agricultural Economics
20. Sustainability and Ecology
21. Agricultural History
22. Pests and Weeds Management
23. Soil Science
24. Agricultural Engineering
25. Animal Science
26. Crop Production
27. Plant Biochemistry
28. Forestry
29. Rural Sociology
30. Horticulture and Ornamental Plants
31. Plant Taxonomy and Scientific Naming
32. Food Scien

## Prompt LLM to attribute a category to each question

In [30]:
prompt_categories = '''Assign to the previous question one category from the list of categories, if in doubt between categories, choose the more specific option. Your answer should start with one line containing only the category name.'''
prompt_categories_2 = '''Assign one category from the provided list to the previous question. If unsure, select the more specific category that best fits. Begin your response with a single line containing only the category name'''

In [149]:
categories_dict = {c: [] for c in categories}
extra_dict = {'suggestions' : [], 'catastrophies' : []}

end_prompt =  'Assign a category to the question above. Your answer should include only the with a single line containing only the name of the chosen category.'

dataset_light = sample(list(dataset), 200)
dataset_all = list(dataset)

for doc in tqdm(dataset_all):
    response = await chat(get_prompt(doc, end_prompt, ''), None, system_prompt=system_prompt)
    try:
        category = response.split('\n')[0].strip()
        if category not in categories:
            print('Suggestion: ', response)
            extra_dict['suggestions'].append((response,doc))
            category = response.split('\n')[-1].strip()
            if category not in categories:
                print('Category not present')
                print(f"Category: {category} | Question: {doc['question']} | Options: {doc['options']}")
        #categories.add(category)
        categories_dict[category].append(doc)
    except:
        extra_dict['catastrophies'].append((response,doc))
        print('Catastrophy', response)
        print(f"Question: {doc['question']} | Options: {doc['options']}")

  5%|▍         | 220/4564 [02:11<40:54,  1.77it/s] 

Suggestion:  Pest and Weeds Management
Category not present
Category: Pest and Weeds Management | Question: Active ingredient in pendimethalin is? | Options: ['33%', '30%', '27%', '35%']
Catastrophy Pest and Weeds Management
Question: Active ingredient in pendimethalin is? | Options: ['33%', '30%', '27%', '35%']


  5%|▌         | 236/4564 [02:20<44:59,  1.60it/s]

Suggestion:  Plant Nutrition or more specifically Crop Nutrient Management
Category not present
Category: Plant Nutrition or more specifically Crop Nutrient Management | Question: Little leaf of Litchi? | Options: ['Zn', 'Mn', 'Mg', 'Cu', 'Al']
Catastrophy Plant Nutrition or more specifically Crop Nutrient Management
Question: Little leaf of Litchi? | Options: ['Zn', 'Mn', 'Mg', 'Cu', 'Al']


  8%|▊         | 376/4564 [03:35<1:32:18,  1.32s/it]

...

 98%|█████████▊| 4467/4564 [44:35<01:57,  1.22s/it]

Suggestion:  Plant Nutrition or more specifically Crop Nutrient Management
Category not present
Category: Plant Nutrition or more specifically Crop Nutrient Management | Question: Criteria of essentiality by given by: | Options: ['Julis Von Suches', 'Arnon', 'Jean Baptist', 'Liegbig']
Catastrophy Plant Nutrition or more specifically Crop Nutrient Management
Question: Criteria of essentiality by given by: | Options: ['Julis Von Suches', 'Arnon', 'Jean Baptist', 'Liegbig']


 98%|█████████▊| 4473/4564 [44:41<01:37,  1.07s/it]

Suggestion:  Plant Nutrition or more specifically Crop Nutrient Management
Category not present
Category: Plant Nutrition or more specifically Crop Nutrient Management | Question: Who among the following presented the theory of essential mineral nutrients in plants?  | Options: ['Aristotle', 'Arnon and stout', 'Liebig', 'Carl Linnaeus']
Catastrophy Plant Nutrition or more specifically Crop Nutrient Management
Question: Who among the following presented the theory of essential mineral nutrients in plants?  | Options: ['Aristotle', 'Arnon and stout', 'Liebig', 'Carl Linnaeus']


100%|██████████| 4564/4564 [45:57<00:00,  1.66it/s]


In [177]:
from collections import Counter

def print_keys_by_length(categories_dict):
    """
    Prints the keys and lengths of their corresponding values from a dictionary,
    in decreasing order of length.

    Args:
        categories_dict (dict): A dictionary where the values are iterable.
    """
    # Create a list of tuples (key, length of value)
    lengths = [(k, len(v)) for k, v in categories_dict.items()]
    
    # Sort the list by length (second item in tuple) in descending order
    sorted_lengths = sorted(lengths, key=lambda x: x[1], reverse=True)

    # Print the sorted keys and lengths
    for k, length in sorted_lengths:
        print(k, length)

print_keys_by_length(categories_dict)

Genetics, Breeding and Seeds Management 745
Soil Science 388
Horticulture and Ornamental Plants 377
Plant Pathology 347
Crop Production 340
Plant Physiology 317
Crop Nutrient Management 221
Plant Science 192
Animal Science 173
Agricultural Economics 166
Entomology 155
Plant Taxonomy and Scientific Naming 141
Irrigation and Water Management 114
Agricultural Geography 99
Pests and Weeds Management 88
Microbiology 79
Agricultural Education 74
Plant Anatomy 73
Fertilizers 67
Agricultural Engineering 66
Plant Biochemistry 52
Agricultural History 50
Agricultural Extension 47
Food Science 38
Climate and Agriculture 37
Plant Protection 31
Farm Management 30
Sustainability, Forestry and Ecology 25
Precision Agriculture 16


In [178]:
len(categories_dict)

29

In [175]:
# Combine the lists from 'key1' and 'key2' into a new key 'key_new'

def combine_keys(new_key,key1,key2):
    categories_dict[new_key] = categories_dict[key1] + categories_dict[key2]
    
    #categories_dict.pop(key1)
    categories_dict.pop(key2)

In [179]:
save_to_json(categories_dict, '/workdir/category_dictionary.json')

In [182]:
for doc in sample(categories_dict['Precision Agriculture'],16):
    print(f"Question: {doc['question']} | Options: {doc['options']}")

Question: Which system provides the accurate positional information, which will be more useful in location the spatial variability with accuracy?  | Options: ['DSS', 'GPS', 'GIS', 'All of these']
Question: Application of Remote sensing and GIS based technologies are essential for? | Options: ['Integrated Farming', 'Intensive Farming', 'Precision Farming', 'Mixed Farming']
Question: Which system provides the accurate positional information, which will be more useful in locating the spatial variability with accuracy? | Options: ['GPS', 'GIS', 'DSS', 'All of these']
Question: Which one of the following techniques is used to monitor crop health? | Options: ['Surveillance', 'Vigilance', 'Remote sensing', 'All of these']
Question: Development of precision GIS / GPS auto negative systems increased the efficiency of field operations in ……………………….? | Options: ['Processing farming', 'Conservation agriculture', 'Cooperative farming', 'Multistorey farming']
Question: Which software is used for spa